!pip install requests

In [ ]:
import requests
import json
import time

In [ ]:
### TEST API
ploads = {'gameid':440,'format':'json'}
r = requests.get('http://api.steampowered.com/ISteamUserStats/GetGlobalAchievementPercentagesForApp/v0002',params=ploads)
print(r.text)
print(r.url)

{"achievementpercentages":{"achievements":[{"name":"TF_SCOUT_LONG_DISTANCE_RUNNER","percent":53.0999984741210938},{"name":"TF_HEAVY_DAMAGE_TAKEN","percent":43.2999992370605469},{"name":"TF_GET_CONSECUTIVEKILLS_NODEATHS","percent":39.7000007629394531},{"name":"TF_PYRO_CAMP_POSITION","percent":32.9000015258789063},{"name":"TF_KILL_NEMESIS","percent":30.5},{"name":"TF_BURN_PLAYERSINMINIMUMTIME","percent":30.5},{"name":"TF_PYRO_BURN_MEDICPAIR","percent":30},{"name":"TF_SNIPER_KILL_MIDAIR_SCOUT","percent":28.6000003814697266},{"name":"TF_DEMOMAN_BOUNCE_AND_KILL","percent":28.2000007629394531},{"name":"TF_PYRO_KILL_TEAMWORK","percent":28.1000003814697266},{"name":"TF_HEAVY_KILL_MIDAIR_MINIGUN","percent":27.8999996185302734},{"name":"TF_PYRO_DOUBLE_KO","percent":27.8999996185302734},{"name":"TF_ENGINEER_DISPENSER_HEAL_GROUP","percent":27.6000003814697266},{"name":"TF_HEAVY_TAKE_MULTI_DAMAGE","percent":27.5},{"name":"TF_HALLOWEEN_DOMINATE_FOR_HAT","percent":26.6000003814697266},{"name":"TF_SNI

In [ ]:
### GET STEAMIDS

def getFriendList(steamid):
  ploads = {'steamid':steamid,'format':'json', 'relationship':'all', 'key': '92B91BC3A28D928DF3AE65CB3B75943D'}
  r = requests.get('http://api.steampowered.com/ISteamUser/GetFriendList/v0001',params=ploads)
  return r.json()

#On part de mon propre steamId
steamIdList = ['76561198006667424']

#Puis on récupère les steamId de ma liste d'amis, et ensuite ceux des amis de mes amis, ...
j=0
for i in range(1000):
  dic = getFriendList(steamIdList[j])

  if bool(dic)!=False and 'friendslist' in dic.keys():
    for k in range(len(dic['friendslist']['friends'])):
      newSteamId = dic['friendslist']['friends'][k]['steamid']
      if not (newSteamId in steamIdList):
        steamIdList.append(newSteamId)

  if j<len(steamIdList)-1: 
    j+=1
    print("j="+str(j)+"\n")
    print("len(steamIdList)="+str(len(steamIdList))+"\n")

print(len(steamIdList))

In [ ]:
#Enregistrement de la liste des steamIds
jsonString = json.dumps(steamIdList)

with open('SteamIdList.txt', 'w') as f:
    f.write(jsonString)

print(steamIdList)


In [ ]:
#Lecture de la liste des steamIds à partir du fichier texte
with open('SteamIdList.txt', 'r') as f:
    steamIds = f.read()
    steamIds = json.loads(steamIds)
    #print(steamIds[0])
print(len(steamIds))

76421


In [ ]:
### GET GAME INFO FROM STEAMID
def getGameInfo(steamId):
  ploads = {'steamid': steamId,'format':'json', 'key': '92B91BC3A28D928DF3AE65CB3B75943D'}
  r = requests.get('http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001',params=ploads)
  if (
      r.status_code != 204 and
      r.headers["content-type"].strip().startswith("application/json")
  ):
    return r.json()['response']

In [ ]:
#Test de la fonction getGameInfo
json_test = getGameInfo('76561198006667424')
print(json_test)


{'steamid': '76561198006667424', 'game_count': 330, 'games': [{'appid': 240, 'playtime_forever': 2677, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0, 'rtime_last_played': 1521840028}, {'appid': 4000, 'playtime_forever': 57279, 'playtime_windows_forever': 1080, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0, 'rtime_last_played': 1651526442}, {'appid': 4760, 'playtime_forever': 19541, 'playtime_windows_forever': 789, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0, 'rtime_last_played': 1633803888}, {'appid': 4770, 'playtime_forever': 1720, 'playtime_windows_forever': 6, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0, 'rtime_last_played': 1592319135}, {'appid': 10500, 'playtime_forever': 35248, 'playtime_windows_forever': 4279, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0, 'rtime_last_played': 1636310856}, {'appid': 23420, 'playtime_forever': 233, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_li

In [ ]:
#Fonction pour enregistrer le dataset dans un fichier json
def saveData(jsonData):
  jsonString = json.dumps(jsonData)

  with open('SteamGameData.json', 'w') as f:
      f.write(jsonString)

In [ ]:
#Lecture du fichier json dataset
with open('SteamGameData.json', 'r') as f:
    dataset = f.read()
    dataset = json.loads(dataset)
    print(len(dataset))

print(dataset[0])

1341741
{'steamid': '76561198006667424', 'appid': 240, 'total_playtime': 2677}


In [ ]:
#Constitution du dataset à partir de la liste des steamIds
#On scrap l'API par paquets de 5000 steamIds sous peine de se faire blacklist l'adresse IP

null_line = {'game_count': 0, 'games': []}
i_last = 20000
print(len(steamIds))

for i in range(i_last, i_last + 5000):
  new_json_line = getGameInfo(steamIds[i])
  time.sleep(0.05)
  if bool(new_json_line)!=False:
    new_json_line = {**{'steamid': steamIds[i]}, **new_json_line}
    dataset.append(new_json_line)
  #print(new_json_line)
  
print("taille dataset = ",len(dataset))
saveData(dataset)


In [ ]:
#Epurage du dataset en retirant les informations inutiles

new_dataset = []
print(len(dataset))
for i in range(len(dataset)):
  for j in range(dataset[i]['game_count']):
    steamid = dataset[i]['steamid']
    appid = dataset[i]['games'][j]['appid']
    total_playtime = dataset[i]['games'][j]['playtime_forever']
    new_json_line = {'steamid': steamid, 'appid': appid, 'total_playtime': total_playtime}
    new_dataset.append(new_json_line)

print(len(new_dataset))
print(new_dataset[0])

In [ ]:
saveData(new_dataset)

In [ ]:
#Autre version du dataset en CSV
csv = "steamid; appid; total_playtime\n"
for i in range(len(new_dataset)):
  csv += str(new_dataset[i]['steamid'])+";"+str(new_dataset[i]['appid'])+";"+str(new_dataset[i]['total_playtime'])+"\n"

with open('SteamGameData.csv', 'w') as f:
    f.write(csv)